# 🔗🇨🇳 LangChain核心源代码解读分享

## 课程开始：💡 这节课会带给你

1. 一起阅读 LangChain 源代码：Langchain(Python)和 LangGraph 两个模块
2. 在源码理解基础上，彻底掌控 Runnable 和 LCEL
3. 在源码理解基础上，自己动手集成大模型到 LangChain
4. 在源码理解基础上，自己动手定制智能体

<div class="alert alert-warning">
<b>这节课不会涉及：</b><br>
<ul>
    <li>大模型认知相关概念（请参考AGI课堂正课相关章节）</li>
    <li>Langchain 开发基础（请参考AGI课堂正课相关章节）</li>
    <li>解读 Langchain 中有成熟文档的组件源码（如prompt模板、messages、Document、retriever等）</li>
</ul> 
</div>

## （一）如何解读 Langchain 的源代码结构

### 1、你有哪些资源可以利用？

![](./langchain_ai.png)

（1）学习资源：
- [langchain源代码](https://github.com/langchain-ai/)：All thins here !!!
- [langchain官方文档](https://python.langchain.com/docs)：与源代码相互印证
- [langgraph example](https://github.com/langchain-ai/langgraph/tree/main/examples)：Jupyter Notes
- [本课件在我的个人github上](https://github.com/arcstep)

（2）良师益友：
- langchain [聊天](https://chat.langchain.com/?llm=anthropic_claude_2_1)：免费的大模型+RAG（也可以学习其源码）
- Github Copilot：程序员无法离开的工具，就像现在的人开车无法离开地图导航

（3）动手实践：
- 按最新框架修改旧的官方例子
- 基于最新langchain框架定义自己的使用框架


### 2、❤️ 阅读源码：LangChain 源代码概览

### 3、❤️ 阅读源码：为什么要阅读 LangChain 源码？

- 与最先进的框架一起成长：Langchain 很快会成熟（当前也成熟，但有使用门槛）
- 作为了解大模型生态的捷径
- 作为了解服务集成选型的捷径
- 作为了解AI应用生态的捷径
- 作为了解智能体生态的捷径

### 4、LangChain核心框架的三轮迭代

- Runnable + Chain（指 Chain 类族名称，非指 LCEL 生成的Runnable对象）
- Runnable + LCEL
- Runnable + LCEL + Langgraph

## （二）直接动手：从零开始集成大模型到 Langchain 实例

### 1、❤️ 阅读源码：了解 Langchain 中大模型组件的结构

### 2、✍️ 代码实践：如何实现一个模拟LLM

- 支持 invoke
- 支持 stream
- 支持 astream
- 支持 astream_events

### 3、✍️ 代码实践：如何集成智谱AI大模型？

- 支持 tools-calling
- 支持 function-calling

- 完整实现：[langchain_zhipu](https://github.com/arcstep/langchain_zhipu)

## （三）全面拆解 Runnable 结构

### 1、为什么说可以用 Runnable 组件在生产系统中搭积木？

- Runnable接口标准化的价值
    - 替换大模型：从GPT到国产云模型、到开源自训模型的多层次尝试和落地是必要的
    - 替换调用方式：满足业务验证、生产上线、业务扩容等多场景
    - 替换提示语模板：适应技术研究、业务定制、运营优化等多阶段
    - 替换解析器：适应模块调用、API调用等多协议对接
    - 替换回调集成：langsmith、langfuse云、langfuse本地化、自建运维系统
    - 替换向量数据库：适应不同场景、阶段的技术选型调整
    - 替换持久化：...
    - 替换智能体：...

### 2、❤️ 阅读源码：Runnable 组件8个方法的实现逻辑

### 3、❤️ 阅读源码：Runnable 组件配置自举能力

- 了解Runnable 组件的 8 个方法的默认实现
- 自定义Runnable（必要实现一般是invoke / stream / astream，或对应的内部函数）
- invoke：标准化调度
- batch: 标准化批量调度
- stream：标准化流式输出
- ainvoke / astream / abatch: 标准化异步调度
- astream_log / astream_events: 在链、智能体、langgraph等输出中按照names、tags、events提取流式日志
- config：统一管理配置
- schema：统一探查参数和配置

- 序列化

- 配置自举：在开放式应用中支持客户端自动识别自定义服务
- 容错：标准化重试策略
- 与langserve等api框架标准化对接
- 与langfuse等callbck框架标准化对接
- 与langchainjs等异构实现标准化对接


### 4、遗留的 Chain 是什么？

- langchain中提前写好的Chain资源

- 这些 Chain 局限性在究竟哪里？
    - 流程不灵活
    - 支持流式输出不彻底

### 5、✍️ 代码实践：若干种情况下的流输出

## （四）全面拆解 LCEL 能力

### 1、LCEL 比 遗留 Chain 多哪些优势？

- LCEL 构建的替代 Chain
- 了解支撑LCEL的Runnable组件
    - Lambda
    - 迭代器
    - 字典和并行
    - 路由
    - 条件
    - 迭代执行
    - 绑定
    - 绘图

### 2、如何用 LCEL 定义智能体？

- 工具：定义一个简单工具
- 智能体：
    - Tools-Calling 智能体
    - ReAct 智能体
- 执行器：AgentExecutor

### 3、❤️ 阅读源码：了解 AgentExecutor 的执行逻辑

### 4、✍️ 代码实践：如何用 AgentExcutor 再现《手撕AutoGPT》？

难点：
- 官方例子和内置智能体无法支持pydantic参数解析（智谱AI等推理能力较弱的模型可以使用）

## （五）全面拆解 LangGraph 能力

### 1、LangGraph 比 LCEL 多了什么？

### 2、如何使用 LangGraph 定义智能体？

### 3、❤️ 阅读源码：了解 LangGraph 的执行逻辑

### 4、✍️ 代码实践：如何用 LangGraph 再现《手撕AutoGPT》？

难点：
- 官方例子和内置智能体无法支持流

## 课程结束

### 1、课程总结

- 我们一起月读了langchain的源代码结构和细节
- 我们学习了如何自己动手集成大模型到 langchain 中
- 我们拆解了langchain的基石组件：Runnable
- 我们拆解了langchain的核心逻辑能力：LCEL
- 我们拆解了langchain的最新逻辑能力：langgraph
- 我们动手做了一些代码实践

### 2、最后建议

- 技术选型时要对 Langchain 有绝对信心（几乎都不会是langchain的错）
- 内置链尽量使用LCEL链
- 内置智能体尽量使用 Langgraph
- 自定义智能体时使用 Langgraph
- 模块优先做成Runnable或LCEL链，其次再考虑Lambda
- 工具中包含大模型调用时优先做成Runnable或LCEL链，其次再考虑invoke


## 彩蛋

### 1、✍️ 代码实践：如何同时使用langchain的记忆和持久化

这是 langchain 文档中一个自相矛盾的地方，留给大家课后讨论。

### 2、✍️ 代码实践：如何将自己训的大模型集成到 langchain 中